In [4]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns

# Read data with proper error handling
try:
    deposits = pd.read_csv('/content/Deposit Data.csv', skiprows=2, header=1)
    withdrawals = pd.read_csv('/content/Withdrawal Data.csv', skiprows=2, header=1)
    gameplays = pd.read_csv('/content/User Gameplay data.csv', skiprows=2, header=1, low_memory=False)
    print("✅ Data files loaded successfully")
except Exception as e:
    print(f"❌ Error loading data: {e}")

# Data preprocessing
print("\n" + "="*60)
print("DATA PREPROCESSING")
print("="*60)

# Convert datetime with error handling
for name, df in [('Deposits', deposits), ('Withdrawals', withdrawals), ('Gameplays', gameplays)]:
    try:
        df['Datetime'] = pd.to_datetime(df['Datetime'], format='%d-%m-%Y %H:%M')
        print(f"✅ {name}: DateTime conversion successful")
    except Exception as e:
        print(f"❌ {name}: DateTime conversion failed - {e}")

# Remove duplicates and show impact
for name, df in [('Deposits', deposits), ('Withdrawals', withdrawals), ('Gameplays', gameplays)]:
    before = len(df)
    df.drop_duplicates(inplace=True)
    after = len(df)
    if before > after:
        print(f"🔄 {name}: Removed {before-after} duplicates ({after} records remaining)")
    else:
        print(f"✅ {name}: No duplicates found ({after} records)")

# Assign slots
def get_slot(ts):
    return '$1' if ts.hour < 12 else '$2'  # Changed to $1/$2

for df in [deposits, withdrawals, gameplays]:
    df['date'] = df['Datetime'].dt.date
    df['slot'] = df['Datetime'].apply(get_slot)

# Normalize user ID column names
deposits.rename(columns={'User Id': 'UserID'}, inplace=True)
withdrawals.rename(columns={'User Id': 'UserID'}, inplace=True)
gameplays.rename(columns={'User ID': 'UserID'}, inplace=True)

print(f"\n📊 Data Summary:")
print(f"   • Total Deposits: {len(deposits):,}")
print(f"   • Total Withdrawals: {len(withdrawals):,}")
print(f"   • Total Gameplays: {len(gameplays):,}")
print(f"   • Unique Players: {len(set(deposits['UserID'].unique()) | set(withdrawals['UserID'].unique()) | set(gameplays['UserID'].unique())):,}")

# PART A: SLOT-WISE LOYALTY POINTS CALCULATION
print("\n" + "="*60)
print("PART A: SLOT-WISE LOYALTY POINTS ANALYSIS")
print("="*60)

selected_filters = [
    ('2022-10-02', '$1', '2nd October Slot $1'),
    ('2022-10-16', '$2', '16th October Slot $2'),
    ('2022-10-18', '$1', '18th October Slot $1'),
    ('2022-10-26', '$2', '26th October Slot $2')
]

results = []
detailed_results = {}

for date_str, slot, description in selected_filters:
    print(f"\n📈 Analyzing {description}")
    date = datetime.strptime(date_str, '%Y-%m-%d').date()

    # Filter data for specific date and slot
    d = deposits[(deposits['date'] == date) & (deposits['slot'] == slot)]
    w = withdrawals[(withdrawals['date'] == date) & (withdrawals['slot'] == slot)]
    g = gameplays[(gameplays['date'] == date) & (gameplays['slot'] == slot)]

    # Aggregate by user
    d_sum = d.groupby('UserID')['Amount'].sum().rename('deposit_amt')
    d_count = d.groupby('UserID').size().rename('deposit_count')

    w_sum = w.groupby('UserID')['Amount'].sum().rename('withdraw_amt')
    w_count = w.groupby('UserID').size().rename('withdraw_count')

    g_count = g.groupby('UserID').size().rename('games_played')

    # Combine all metrics
    df = pd.concat([d_sum, d_count, w_sum, w_count, g_count], axis=1).fillna(0)

    # Calculate loyalty points using the given formula
    df['loyalty_points'] = (
        0.01 * df['deposit_amt'] +
        0.005 * df['withdraw_amt'] +
        0.001 * (df['deposit_count'] - df['withdraw_count']).clip(lower=0) +
        0.2 * df['games_played']
    )

    df['date'] = date_str
    df['slot'] = slot
    df['description'] = description

    # Store detailed results
    detailed_results[description] = df.copy()

    # Summary statistics
    active_players = len(df[df['loyalty_points'] > 0])
    avg_loyalty = df['loyalty_points'].mean()
    top_player = df['loyalty_points'].idxmax() if len(df) > 0 else 'None'
    max_points = df['loyalty_points'].max() if len(df) > 0 else 0

    print(f"   • Active Players: {active_players}")
    print(f"   • Average Loyalty Points: {avg_loyalty:.2f}")
    print(f"   • Top Player: {top_player} ({max_points:.2f} points)")
    print(f"   • Total Deposits: ₹{d['Amount'].sum():,.2f}")
    print(f"   • Total Withdrawals: ₹{w['Amount'].sum():,.2f}")
    print(f"   • Total Games: {len(g)}")

    results.append(df.reset_index())

part_a_result = pd.concat(results, ignore_index=True)

# PART A: DETAILED ANALYSIS FOR EACH SLOT
print(f"\n📋 TOP 5 PLAYERS FOR EACH SLOT:")
for description in detailed_results:
    print(f"\n{description}:")
    top_5 = detailed_results[description].nlargest(5, 'loyalty_points')[['loyalty_points', 'deposit_amt', 'withdraw_amt', 'games_played']]
    if len(top_5) > 0:
        for idx, (user_id, row) in enumerate(top_5.iterrows(), 1):
            print(f"   {idx}. Player {user_id}: {row['loyalty_points']:.2f} points (₹{row['deposit_amt']:.0f} deposit, ₹{row['withdraw_amt']:.0f} withdrawal, {row['games_played']:.0f} games)")
    else:
        print("   No active players in this slot")

# PART A: MONTHLY ANALYSIS
print("\n" + "="*60)
print("PART A: MONTHLY ANALYSIS (OCTOBER 2022)")
print("="*60)

oct_start, oct_end = '2022-10-01', '2022-10-31'
d_month = deposits[(deposits['Datetime'] >= oct_start) & (deposits['Datetime'] <= oct_end)]
w_month = withdrawals[(withdrawals['Datetime'] >= oct_start) & (withdrawals['Datetime'] <= oct_end)]
g_month = gameplays[(gameplays['Datetime'] >= oct_start) & (gameplays['Datetime'] <= oct_end)]

print(f"📊 October 2022 Data Summary:")
print(f"   • Total Deposits: {len(d_month):,} transactions, ₹{d_month['Amount'].sum():,.2f}")
print(f"   • Total Withdrawals: {len(w_month):,} transactions, ₹{w_month['Amount'].sum():,.2f}")
print(f"   • Total Games: {len(g_month):,}")

# Monthly aggregations
d_sum = d_month.groupby('UserID')['Amount'].sum().rename('deposit_amt')
d_count = d_month.groupby('UserID').size().rename('deposit_count')
w_sum = w_month.groupby('UserID')['Amount'].sum().rename('withdraw_amt')
w_count = w_month.groupby('UserID').size().rename('withdraw_count')
g_count = g_month.groupby('UserID').size().rename('games_played')

monthly_df = pd.concat([d_sum, d_count, w_sum, w_count, g_count], axis=1).fillna(0)
monthly_df['loyalty_points'] = (
    0.01 * monthly_df['deposit_amt'] +
    0.005 * monthly_df['withdraw_amt'] +
    0.001 * (monthly_df['deposit_count'] - monthly_df['withdraw_count']).clip(lower=0) +
    0.2 * monthly_df['games_played']
)

# Ranking (loyalty points first, then games played for tie-breaking)
monthly_df = monthly_df.sort_values(['loyalty_points', 'games_played'], ascending=False)
monthly_df['rank'] = range(1, len(monthly_df) + 1)

print(f"\n🏆 TOP 10 PLAYERS (OCTOBER 2022):")
top_10 = monthly_df.head(10)
for idx, (user_id, row) in enumerate(top_10.iterrows(), 1):
    print(f"   {idx:2d}. Player {user_id}: {row['loyalty_points']:.2f} points | "
          f"₹{row['deposit_amt']:.0f} deposits | ₹{row['withdraw_amt']:.0f} withdrawals | "
          f"{row['games_played']:.0f} games")

# PART A: STATISTICAL ANALYSIS
print(f"\n📈 STATISTICAL ANALYSIS:")

# Average deposit amount (Question 3)
avg_deposit = d_month['Amount'].mean()
print(f"   • Average Deposit Amount: ₹{avg_deposit:.2f}")

# Average deposit amount per user per month (Question 4)
avg_deposit_per_user = d_month.groupby('UserID')['Amount'].sum().mean()
print(f"   • Average Deposit per User (Monthly): ₹{avg_deposit_per_user:.2f}")

# Average games played per user (Question 5)
avg_games_per_user = g_month.groupby('UserID').size().mean()
print(f"   • Average Games per User (Monthly): {avg_games_per_user:.1f}")

# Additional insights
total_users = len(monthly_df)
active_depositors = len(d_month['UserID'].unique())
active_players = len(g_month['UserID'].unique())

print(f"   • Total Users with Activity: {total_users:,}")
print(f"   • Users who Deposited: {active_depositors:,} ({active_depositors/total_users*100:.1f}%)")
print(f"   • Users who Played Games: {active_players:,} ({active_players/total_users*100:.1f}%)")

# PART B: BONUS ALLOCATION
print("\n" + "="*60)
print("PART B: BONUS ALLOCATION STRATEGY")
print("="*60)

top_50 = monthly_df.head(50).copy()

# Method 1: Pure Proportional Distribution
top_50['bonus_proportional'] = (top_50['loyalty_points'] / top_50['loyalty_points'].sum()) * 50000
top_50['bonus_proportional'] = top_50['bonus_proportional'].round(2)

# Method 2: Hybrid Distribution (₹400 base + proportional)
base_bonus = 400
performance_pool = 50000 - (base_bonus * 50)
top_50['bonus_hybrid'] = base_bonus + (top_50['loyalty_points'] / top_50['loyalty_points'].sum()) * performance_pool
top_50['bonus_hybrid'] = top_50['bonus_hybrid'].round(2)

# Method 3: Tiered Distribution
def assign_tier_bonus(rank):
    if rank <= 5:
        return 2000
    elif rank <= 15:
        return 1500
    elif rank <= 35:
        return 1000
    else:
        return 333.33

top_50['bonus_tiered'] = top_50['rank'].apply(assign_tier_bonus)

print(f"💰 BONUS ALLOCATION COMPARISON:")
print(f"   • Total Pool: ₹50,000")
print(f"   • Recipients: Top 50 Players")
print(f"\n   Method Comparison:")
print(f"   • Proportional - Range: ₹{top_50['bonus_proportional'].min():.2f} to ₹{top_50['bonus_proportional'].max():.2f}")
print(f"   • Hybrid - Range: ₹{top_50['bonus_hybrid'].min():.2f} to ₹{top_50['bonus_hybrid'].max():.2f}")
print(f"   • Tiered - Range: ₹{top_50['bonus_tiered'].min():.2f} to ₹{top_50['bonus_tiered'].max():.2f}")

print(f"\n🏅 TOP 10 BONUS RECIPIENTS:")
for idx, (user_id, row) in enumerate(top_50.head(10).iterrows(), 1):
    print(f"   {idx:2d}. Player {user_id}: ₹{row['bonus_hybrid']:.2f} "
          f"({row['loyalty_points']:.2f} points)")

# PART C: FORMULA ANALYSIS
print("\n" + "="*60)
print("PART C: LOYALTY FORMULA FAIRNESS ANALYSIS")
print("="*60)

# Analyze contribution of each component
monthly_df['deposit_contribution'] = 0.01 * monthly_df['deposit_amt']
monthly_df['withdrawal_contribution'] = 0.005 * monthly_df['withdraw_amt']
monthly_df['frequency_contribution'] = 0.001 * (monthly_df['deposit_count'] - monthly_df['withdraw_count']).clip(lower=0)
monthly_df['games_contribution'] = 0.2 * monthly_df['games_played']

# Calculate percentages for top 50
top_50_analysis = monthly_df.head(50)
total_points = top_50_analysis['loyalty_points'].sum()

print(f"🔍 LOYALTY POINTS COMPOSITION ANALYSIS (Top 50 Players):")
print(f"   • Deposit Points: {top_50_analysis['deposit_contribution'].sum()/total_points*100:.1f}% of total")
print(f"   • Withdrawal Points: {top_50_analysis['withdrawal_contribution'].sum()/total_points*100:.1f}% of total")
print(f"   • Frequency Points: {top_50_analysis['frequency_contribution'].sum()/total_points*100:.1f}% of total")
print(f"   • Games Points: {top_50_analysis['games_contribution'].sum()/total_points*100:.1f}% of total")

# Identify potential issues
withdrawal_beneficiaries = len(monthly_df[monthly_df['withdrawal_contribution'] > monthly_df['deposit_contribution']])
games_dominance = len(monthly_df[monthly_df['games_contribution'] > (monthly_df['deposit_contribution'] + monthly_df['withdrawal_contribution'])])

print(f"\n⚠️  POTENTIAL FORMULA ISSUES:")
print(f"   • Players earning more from withdrawals than deposits: {withdrawal_beneficiaries}")
print(f"   • Players where games dominate financial contributions: {games_dominance}")

# Summary statistics for report
print(f"\n📋 SUMMARY FOR REPORT:")
print(f"   • Total Active Users (October): {len(monthly_df):,}")
print(f"   • Average Loyalty Points: {monthly_df['loyalty_points'].mean():.2f}")
print(f"   • Loyalty Points Range: {monthly_df['loyalty_points'].min():.2f} - {monthly_df['loyalty_points'].max():.2f}")
print(f"   • Standard Deviation: {monthly_df['loyalty_points'].std():.2f}")

# Export results for further analysis
print(f"\n💾 EXPORT READY:")
print(f"   • part_a_result: Slot-wise analysis results")
print(f"   • monthly_df: Complete monthly rankings")
print(f"   • top_50: Top 50 players with bonus calculations")
print(f"   • detailed_results: Detailed slot-wise breakdowns")

# Optional: Save to CSV files
try:
    part_a_result.to_csv('slot_wise_analysis.csv', index=False)
    monthly_df.to_csv('monthly_rankings.csv', index=True)
    top_50.to_csv('top_50_bonus_allocation.csv', index=True)
    print(f"✅ Results exported to CSV files")
except:
    print(f"ℹ️  CSV export skipped (optional)")

print(f"\n" + "="*60)
print("ANALYSIS COMPLETE ✅")
print("="*60)

✅ Data files loaded successfully

DATA PREPROCESSING
✅ Deposits: DateTime conversion successful
✅ Withdrawals: DateTime conversion successful
✅ Gameplays: DateTime conversion successful
🔄 Deposits: Removed 1 duplicates (17437 records remaining)
✅ Withdrawals: No duplicates found (3566 records)
🔄 Gameplays: Removed 13736 duplicates (341530 records remaining)

📊 Data Summary:
   • Total Deposits: 17,437
   • Total Withdrawals: 3,566
   • Total Gameplays: 341,530
   • Unique Players: 1,000

PART A: SLOT-WISE LOYALTY POINTS ANALYSIS

📈 Analyzing 2nd October Slot $1
   • Active Players: 636
   • Average Loyalty Points: 35.73
   • Top Player: 634 (1478.36 points)
   • Total Deposits: ₹1,397,390.00
   • Total Withdrawals: ₹1,525,167.00
   • Total Games: 5617

📈 Analyzing 16th October Slot $2
   • Active Players: 594
   • Average Loyalty Points: 35.49
   • Top Player: 634 (1491.56 points)
   • Total Deposits: ₹1,430,743.00
   • Total Withdrawals: ₹1,138,745.00
   • Total Games: 5413

📈 Analyzi